# LoRA 样例脚本

## 第一步：安装

安装 Data-Juicer 和 DiffSynth-Studio

In [1]:
import os

In [2]:
!pip install nlpaug

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple


In [3]:
!pip install --upgrade pip
!pip install timm==0.5.4

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple


In [4]:
!pip install simple-aesthetics-predictor

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple


In [5]:
!git clone https://github.com/modelscope/data-juicer.git

fatal: 目标路径 'data-juicer' 已经存在，并且不是一个空目录。


In [6]:
!pip install -v -e data-juicer

Using pip 24.2 from /usr/local/lib/python3.10/site-packages/pip (python 3.10)
Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
Obtaining file:///mnt/workspace/data-juicer
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info
  writing /tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/dependency_links.txt
  writing entry points to /tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/entry_points.txt
  writing requirements to /tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/SOURCES.txt'
  reading manifest file '/tmp/pip-pip-egg-info-ssq4_cis/py_data_juicer.egg-info/SOURCES.txt'
  adding license file 'LICE

In [7]:
!pip uninstall pytorch-lightning -y

Found existing installation: pytorch-lightning 2.4.0
Uninstalling pytorch-lightning-2.4.0:
  Successfully uninstalled pytorch-lightning-2.4.0


In [8]:
!pip install peft lightning pandas torchvision

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
  Using cached https://mirrors.cloud.aliyuncs.com/pypi/packages/2b/d2/ecd65ff1e0b1ca79f9785dd65d5ced7ec2643a828068aaa24e47e4c84a14/pytorch_lightning-2.4.0-py3-none-any.whl (815 kB)


In [9]:
!git clone https://github.com/modelscope/DiffSynth-Studio.git

fatal: 目标路径 'DiffSynth-Studio' 已经存在，并且不是一个空目录。


In [10]:
!pip install -e DiffSynth-Studio

Looking in indexes: https://mirrors.cloud.aliyuncs.com/pypi/simple
Obtaining file:///mnt/workspace/DiffSynth-Studio
  Preparing metadata (setup.py) ... done
  Attempting uninstall: diffsynth
    Found existing installation: diffsynth 1.0.0
    Uninstalling diffsynth-1.0.0:
      Successfully uninstalled diffsynth-1.0.0
  DEPRECATION: Legacy editable install of diffsynth==1.0.0 from file:///mnt/workspace/DiffSynth-Studio (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for diffsynth


# 请在这里手动重启 Notebook kernel

## 第二步：下载数据集

In [11]:
from modelscope.msdatasets import MsDataset

ds = MsDataset.load(
    'AI-ModelScope/lowres_anime',
    subset_name='default',
    split='train',
    cache_dir="/mnt/workspace/data"
)

/usr/local/lib/python3.10/site-packages/datasets/builder.py:883: FutureWarning: 'try_from_hf_gcs' was deprecated in version 2.16.0 and will be removed in 3.0.0.
  warnings.warn(


保存数据集中的图片及元数据

In [12]:
import json, os
from data_juicer.utils.mm_utils import SpecialTokens
from tqdm import tqdm


os.makedirs("./data/lora_dataset/train", exist_ok=True)
os.makedirs("./data/data-juicer/input", exist_ok=True)
with open("./data/data-juicer/input/metadata.jsonl", "w") as f:
    for data_id, data in enumerate(tqdm(ds)):
        image = data["image"].convert("RGB")
        image.save(f"/mnt/workspace/data/lora_dataset/train/{data_id}.jpg")
        metadata = {"text": "二次元", "image": [f"/mnt/workspace/data/lora_dataset/train/{data_id}.jpg"]}
        f.write(json.dumps(metadata))
        f.write("\n")

100%|██████████| 1454/1454 [00:23<00:00, 63.10it/s] 


## 第三步：数据处理

使用 data-juicer 处理数据

In [13]:
data_juicer_config = """
# global parameters
project_name: 'data-process'
dataset_path: './data/data-juicer/input/metadata.jsonl'  # path to your dataset directory or file
np: 4  # number of subprocess to process your dataset

text_keys: 'text'
image_key: 'image'
image_special_token: '<__dj__image>'

export_path: './data/data-juicer/output/result.jsonl'

# process schedule
# a list of several process operators with their arguments
process:
    - image_shape_filter:
        min_width: 1024
        min_height: 1024
        any_or_all: any
    - image_aspect_ratio_filter:
        min_ratio: 0.5
        max_ratio: 2.0
        any_or_all: any
"""
with open("data/data-juicer/data_juicer_config.yaml", "w") as file:
    file.write(data_juicer_config.strip())

!dj-process --config data/data-juicer/data_juicer_config.yaml

2024-10-24 22:45:28 | INFO     | data_juicer.config.config:635 - Back up the input config file [/mnt/workspace/data/data-juicer/data_juicer_config.yaml] into the work_dir [/mnt/workspace/data/data-juicer/output]
2024-10-24 22:45:28 | INFO     | data_juicer.config.config:657 - Configuration table: 
╒══════════════════════════╤═════════════════════════════════════════════════════════════════════════╕
│ key                      │ values                                                                  │
╞══════════════════════════╪═════════════════════════════════════════════════════════════════════════╡
│ config                   │ [Path_fr(data/data-juicer/data_juicer_config.yaml, cwd=/mnt/workspace)] │
├──────────────────────────┼─────────────────────────────────────────────────────────────────────────┤
│ hpo_config               │ None                                                                    │
├──────────────────────────┼───────────────────────────────────────────────────────

保存处理好的数据

In [14]:
import pandas as pd
import os, json
from PIL import Image
from tqdm import tqdm


texts, file_names = [], []
os.makedirs("./data/lora_dataset_processed/train", exist_ok=True)
with open("./data/data-juicer/output/result.jsonl", "r") as file:
    for data_id, data in enumerate(tqdm(file.readlines())):
        data = json.loads(data)
        text = data["text"]
        texts.append(text)
        image = Image.open(data["image"][0])
        image_path = f"./data/lora_dataset_processed/train/{data_id}.jpg"
        image.save(image_path)
        file_names.append(f"{data_id}.jpg")
data_frame = pd.DataFrame()
data_frame["file_name"] = file_names
data_frame["text"] = texts
data_frame.to_csv("./data/lora_dataset_processed/train/metadata.csv", index=False, encoding="utf-8-sig")
data_frame

100%|██████████| 129/129 [00:04<00:00, 31.16it/s]


,file_name,text
0,0.jpg,二次元
1,1.jpg,二次元
2,2.jpg,二次元
3,3.jpg,二次元
4,4.jpg,二次元
...,...,...
124,124.jpg,二次元
125,125.jpg,二次元
126,126.jpg,二次元
127,127.jpg,二次元


## 第四步：训练模型

下载模型

In [15]:
from diffsynth import download_models

download_models(["Kolors", "SDXL-vae-fp16-fix"])

2024-10-24 22:45:50.684709: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 22:45:51.278252: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 22:45:52.401044: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Failed to load cpm_kernels:No module named 'cpm_kernels'


    Start downloading models/kolors/Kolors/text_encoder/config.json


    Start downloading models/kolors/Kolors/text_encoder/pytorch_model.bin.index.json


    Start downloading models/kolors/Kolors/text_encoder/pytorch_model-00001-of-00007.bin


    Start downloading models/kolors/Kolors/text_encoder/pytorch_model-00002-of-00007.bin


...

    Start downloading models/kolors/Kolors/text_encoder/pytorch_model-00007-of-00007.bin


    Start downloading models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors


    Start downloading models/kolors/Kolors/vae/diffusion_pytorch_model.safetensors


    Start downloading models/sdxl-vae-fp16-fix/diffusion_pytorch_model.safetensors


['models/kolors/Kolors/text_encoder',
 'models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors',
 'models/kolors/Kolors/vae/diffusion_pytorch_model.safetensors',
 'models/sdxl-vae-fp16-fix/diffusion_pytorch_model.safetensors']

查看训练脚本的输入参数

In [16]:
!python DiffSynth-Studio/examples/train/kolors/train_kolors_lora.py -h

2024-10-24 22:49:27.245024: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 22:49:27.316670: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 22:49:28.461255: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Failed to load cpm_kernels:No module named 'cpm_kernels'
usage: train_kolors_lora.py [-h] --pretrained_unet_path PRETRAINED_UNET_PATH
                            --pretrained_text_encoder_path
                            PRETRAINED_TEXT_ENCODER_PATH


开始训练

提示：

1. 在训练命令中填入 `--modelscope_model_id xxxxx` 以及 `--modelscope_access_token xxxxx` 后，训练程序会在结束时自动上传模型到 ModelScope
2. 部分参数可根据实际需求调整，例如 `lora_rank` 可以控制 LoRA 模型的参数量

In [17]:
import os

cmd = """
python DiffSynth-Studio/examples/train/kolors/train_kolors_lora.py \
  --pretrained_unet_path models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors \
  --pretrained_text_encoder_path models/kolors/Kolors/text_encoder \
  --pretrained_fp16_vae_path models/sdxl-vae-fp16-fix/diffusion_pytorch_model.safetensors \
  --lora_rank 16 \
  --lora_alpha 4.0 \
  --dataset_path data/lora_dataset_processed \
  --output_path ./models \
  --max_epochs 1 \
  --center_crop \
  --use_gradient_checkpointing \
  --precision "16-mixed"
""".strip()

os.system(cmd)

2024-10-24 22:49:34.123191: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-24 22:49:34.165255: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-24 22:49:34.913751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Failed to load cpm_kernels:No module named 'cpm_kernels'


Loading models from: models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors
    model_name: sdxl_unet model_class: SDXLUNet
        This model is initialized with extra kwargs: {'is_kolors': True}
    The following models are loaded: ['sdxl_unet'].
Loading models from: models/kolors/Kolors/text_encoder


Loading checkpoint shards: 100%|██████████| 7/7 [00:01<00:00,  5.49it/s]
/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIB

    The following models are loaded: ['kolors_text_encoder'].
Loading models from: models/sdxl-vae-fp16-fix/diffusion_pytorch_model.safetensors
    model_name: sdxl_vae_encoder model_class: SDXLVAEEncoder
        This model is initialized with extra kwargs: {'upcast_to_float32': True}
    model_name: sdxl_vae_decoder model_class: SDXLVAEDecoder
        This model is initialized with extra kwargs: {'upcast_to_float32': True}
    The following models are loaded: ['sdxl_vae_encoder', 'sdxl_vae_decoder'].
No sdxl_text_encoder models available.
No sdxl_text_encoder_2 models available.
Using kolors_text_encoder from models/kolors/Kolors/text_encoder.
Using sdxl_unet from models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors.
Using sdxl_vae_decoder from models/sdxl-vae-fp16-fix/diffusion_pytorch_model.safetensors.
Using sdxl_vae_encoder from models/sdxl-vae-fp16-fix/diffusion_pytorch_model.safetensors.
No sdxl_ipadapter models available.
No sdxl_ipadapter_clip_image_encoder models ava

/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


Epoch 0: 100%|██████████| 500/500 [09:07<00:00,  0.91it/s, v_num=0, train_loss=0.196]   

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 500/500 [09:07<00:00,  0.91it/s, v_num=0, train_loss=0.196]


0

加载模型

In [18]:
from diffsynth import ModelManager, SDXLImagePipeline
from peft import LoraConfig, inject_adapter_in_model
import torch


def load_lora(model, lora_rank, lora_alpha, lora_path):
    lora_config = LoraConfig(
        r=lora_rank,
        lora_alpha=lora_alpha,
        init_lora_weights="gaussian",
        target_modules=["to_q", "to_k", "to_v", "to_out"],
    )
    model = inject_adapter_in_model(lora_config, model)
    state_dict = torch.load(lora_path, map_location="cpu")
    model.load_state_dict(state_dict, strict=False)
    return model


# Load models
model_manager = ModelManager(torch_dtype=torch.float16, device="cuda",
                             file_path_list=[
                                 "models/kolors/Kolors/text_encoder",
                                 "models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors",
                                 "models/kolors/Kolors/vae/diffusion_pytorch_model.safetensors"
                             ])
pipe = SDXLImagePipeline.from_model_manager(model_manager)

# Load LoRA
pipe.unet = load_lora(
    pipe.unet,
    lora_rank=16, # This parameter should be consistent with that in your training script.
    lora_alpha=2.0, # lora_alpha can control the weight of LoRA.
    lora_path="models/lightning_logs/version_0/checkpoints/epoch=0-step=500.ckpt"
)

Loading models from: models/kolors/Kolors/text_encoder


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

    The following models are loaded: ['kolors_text_encoder'].
Loading models from: models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors
    model_name: sdxl_unet model_class: SDXLUNet
        This model is initialized with extra kwargs: {'is_kolors': True}
    The following models are loaded: ['sdxl_unet'].
Loading models from: models/kolors/Kolors/vae/diffusion_pytorch_model.safetensors
    model_name: sdxl_vae_encoder model_class: SDXLVAEEncoder
        This model is initialized with extra kwargs: {'upcast_to_float32': True}
    model_name: sdxl_vae_decoder model_class: SDXLVAEDecoder
        This model is initialized with extra kwargs: {'upcast_to_float32': True}
    The following models are loaded: ['sdxl_vae_encoder', 'sdxl_vae_decoder'].
No sdxl_text_encoder models available.
No sdxl_text_encoder_2 models available.
Using kolors_text_encoder from models/kolors/Kolors/text_encoder.
Using sdxl_unet from models/kolors/Kolors/unet/diffusion_pytorch_model.safetensors.
Using s

/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


生成图像

In [19]:
torch.manual_seed(0)
image = pipe(
    prompt="未来实验室，一位考古学家，穿着宇航服，正在检查复杂的时间机器，背景是充满高科技设备的实验室，赛博朋克风格, 主要凸显实验室和时间机器，时间机器很大，显得考古学家很小",
    negative_prompt="丑陋、变形、嘈杂、模糊、低对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("1.jpg")
torch.cuda.empty_cache()

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 50/50 [00:20<00:00,  2.44it/s]
/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 1.00 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv2d(input, weight, bias, self.stride,
/usr/local/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU  (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.

In [20]:
torch.manual_seed(1)
image = pipe(
    prompt="古埃及风格，沙漠背景，金字塔建造现场，工人搬运巨石，象形文字装饰，日落时分，沙漠色调，一位考古学家穿着宇航服站在金字塔顶",
    negative_prompt="丑陋、变形、嘈杂、模糊、低对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("2.jpg")
torch.cuda.empty_cache()

100%|██████████| 50/50 [00:18<00:00,  2.70it/s]


In [21]:
torch.manual_seed(2)
image = pipe(
    prompt="唐朝风格，背景是唐朝建筑和园林，展现唐朝宏伟建筑和文化和艺术，身穿宇航服的考古学家站在门前观察",
    negative_prompt="丑陋、变形、嘈杂、模糊、低对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("3.jpg")
torch.cuda.empty_cache()

100%|██████████| 50/50 [00:18<00:00,  2.69it/s]


In [22]:
torch.manual_seed(5)
image = pipe(
    prompt="中世纪欧洲风格，哥特式城堡，一个身穿宇航服的考古学家观察防御战斗，背景是中世纪战争场景，旗帜飘扬，铠甲和剑",
    negative_prompt="丑陋、变形、嘈杂、模糊、低对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("4.jpg")
torch.cuda.empty_cache()

100%|██████████| 50/50 [00:18<00:00,  2.68it/s]


In [23]:
torch.manual_seed(0)
image = pipe(
    prompt="未来世界风格，赛博朋克背景，一位身穿宇航服的考古学家站在高科技的悬空城市中，周围是AI机器人和全息投影等，悬空的建筑和交通，展现AI主导的科技文明",
    negative_prompt="丑陋、变形、嘈杂、模糊、低对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("5.jpg")
torch.cuda.empty_cache()

100%|██████████| 50/50 [00:18<00:00,  2.67it/s]


In [24]:
torch.manual_seed(1)
image = pipe(
    prompt="未来世界与古代文明混合风格，一位身穿宇航服的考古学家站在远古遗迹前，背景是未来城市的天际线，遗迹上有古代象形文字和高科技元素",
    negative_prompt="丑陋、变形、嘈杂、模糊、低对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("6.jpg")
torch.cuda.empty_cache()

100%|██████████| 50/50 [00:18<00:00,  2.67it/s]


In [25]:
torch.manual_seed(7)
image = pipe(
    prompt="未知时代风格为主，混杂很多历史元素，时间机器发出故障报错的的红色警示灯，背景是神秘和未知的氛围，混合了不同历史时期的建筑和服饰，很混杂，一位身穿宇航服的考古学家站在时间机器旁，显得很渺小",
    negative_prompt="丑陋、清晰、高对比度、高清",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("7.jpg")
torch.cuda.empty_cache()

100%|██████████| 50/50 [00:18<00:00,  2.66it/s]


In [26]:
torch.manual_seed(0)
image = pipe(
    prompt="现代与未知时代混合风格，一位渺小的身穿宇航服的考古学家成功修复庞大的时间机器，他挥手告别，背景是现代实验室和未知时代的遗迹，庞大的时间机器发出启动的光芒，展现对历史的新认识",
    negative_prompt="丑陋、变形、嘈杂、模糊、高对比度",
    cfg_scale=4,
    num_inference_steps=50, height=1024, width=1024,
)
image.save("8.jpg")

100%|██████████| 50/50 [00:18<00:00,  2.66it/s]


In [27]:
import numpy as np
from PIL import Image


images = [np.array(Image.open(f"{i}.jpg")) for i in range(1, 9)]
image = np.concatenate([
    np.concatenate(images[0:2], axis=1),
    np.concatenate(images[2:4], axis=1),
    np.concatenate(images[4:6], axis=1),
    np.concatenate(images[6:8], axis=1),
], axis=0)
image = Image.fromarray(image).resize((1024, 2048))
image.save("final.jpg")